In [1]:
import pandas as aju
import numpy as np
import ast
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
credits = aju.read_csv('tmdb_5000_credits.csv')
movies = aju.read_csv('tmdb_5000_movies.csv')

In [3]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [4]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [5]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [6]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [7]:
print(credits.shape,"|",movies.shape)

(4803, 4) | (4803, 20)


In [8]:
df = movies.merge(credits,left_on = 'title',right_on = 'title')
df.shape

(4809, 23)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [10]:
df.loc[0]

budget                                                          237000000
genres                  [{"id": 28, "name": "Action"}, {"id": 12, "nam...
homepage                                      http://www.avatarmovie.com/
id                                                                  19995
keywords                [{"id": 1463, "name": "culture clash"}, {"id":...
original_language                                                      en
original_title                                                     Avatar
overview                In the 22nd century, a paraplegic Marine is di...
popularity                                                     150.437577
production_companies    [{"name": "Ingenious Film Partners", "id": 289...
production_countries    [{"iso_3166_1": "US", "name": "United States o...
release_date                                                   2009-12-10
revenue                                                        2787965087
runtime                               

In [11]:
df = df[['movie_id','title','overview','genres','keywords','cast','crew']]
df.loc[0]

movie_id                                                19995
title                                                  Avatar
overview    In the 22nd century, a paraplegic Marine is di...
genres      [{"id": 28, "name": "Action"}, {"id": 12, "nam...
keywords    [{"id": 1463, "name": "culture clash"}, {"id":...
cast        [{"cast_id": 242, "character": "Jake Sully", "...
crew        [{"credit_id": "52fe48009251416c750aca23", "de...
Name: 0, dtype: object

**WE HAVE GENRES NAME INSIDE A DICTIONARY...SIMILARLY KEYWORDS,CAST,CREW... SO WE NEED TO FETCH THEM FOR OUR PROGRAM**

In [13]:
def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

**THE *convert()* FUNCTION TAKES A STRINGIFIED PYTHON OBJECT (list of dictionaries) AND RETURNS LIST OF 'NAME' VALUES FROM EACH DICTIONARY.**

In [15]:
df['genres'] = df.genres.apply(convert)

In [16]:
df.genres

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4804                        [Action, Crime, Thriller]
4805                                [Comedy, Romance]
4806               [Comedy, Drama, Romance, TV Movie]
4807                                               []
4808                                    [Documentary]
Name: genres, Length: 4809, dtype: object

In [17]:
df['keywords'] = df.keywords.apply(convert)

In [18]:
df.keywords

0       [culture clash, future, space war, space colon...
1       [ocean, drug abuse, exotic island, east india ...
2       [spy, based on novel, secret agent, sequel, mi...
3       [dc comics, crime fighter, terrorist, secret i...
4       [based on novel, mars, medallion, space travel...
                              ...                        
4804    [united states–mexico barrier, legs, arms, pap...
4805                                                   []
4806    [date, love at first sight, narration, investi...
4807                                                   []
4808            [obsession, camcorder, crush, dream girl]
Name: keywords, Length: 4809, dtype: object

In [19]:
df['cast'][0]

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [20]:
df['cast'] = df['cast'].apply(lambda x: [i['name'] for i in ast.literal_eval(x)[:3]])
# Fetching the first 3 names of actors

In [21]:
df.cast

0        [Sam Worthington, Zoe Saldana, Sigourney Weaver]
1           [Johnny Depp, Orlando Bloom, Keira Knightley]
2            [Daniel Craig, Christoph Waltz, Léa Seydoux]
3            [Christian Bale, Michael Caine, Gary Oldman]
4          [Taylor Kitsch, Lynn Collins, Samantha Morton]
                              ...                        
4804    [Carlos Gallardo, Jaime de Hoyos, Peter Marqua...
4805         [Edward Burns, Kerry Bishé, Marsha Dietlein]
4806           [Eric Mabius, Kristin Booth, Crystal Lowe]
4807            [Daniel Henney, Eliza Coupe, Bill Paxton]
4808    [Drew Barrymore, Brian Herzlinger, Corey Feldman]
Name: cast, Length: 4809, dtype: object

In [22]:
df['crew'][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [23]:
df['crew'] = df.crew.apply(lambda x : [i['name'] for i in ast.literal_eval(x) if i['job'] == 'Director'])
# Fetching the director's name

In [24]:
df['crew']

0                                [James Cameron]
1                               [Gore Verbinski]
2                                   [Sam Mendes]
3                            [Christopher Nolan]
4                               [Andrew Stanton]
                          ...                   
4804                          [Robert Rodriguez]
4805                              [Edward Burns]
4806                               [Scott Smith]
4807                               [Daniel Hsia]
4808    [Brian Herzlinger, Jon Gunn, Brett Winn]
Name: crew, Length: 4809, dtype: object

In [25]:
df['tags'] = df['genres'] + df['keywords'] + df['cast'] + df['crew']

In [26]:
df['tags'][0]

['Action',
 'Adventure',
 'Fantasy',
 'Science Fiction',
 'culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d',
 'Sam Worthington',
 'Zoe Saldana',
 'Sigourney Weaver',
 'James Cameron']

In [27]:
df = df[['movie_id','title','overview','tags']]

In [28]:
df

,movie_id,title,overview,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction, ..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action, ocean, drug abuse..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime, spy, based on novel..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller, dc comics, cr..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction, based on ..."
...,...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,"[Action, Crime, Thriller, united states–mexico..."
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,"[Comedy, Romance, Edward Burns, Kerry Bishé, M..."
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","[Comedy, Drama, Romance, TV Movie, date, love ..."
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,"[Daniel Henney, Eliza Coupe, Bill Paxton, Dani..."


In [29]:
df.tags = df.tags.apply(lambda x : " ".join(x))

In [30]:
df.tags

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4804    Action Crime Thriller united states–mexico bar...
4805    Comedy Romance Edward Burns Kerry Bishé Marsha...
4806    Comedy Drama Romance TV Movie date love at fir...
4807    Daniel Henney Eliza Coupe Bill Paxton Daniel Hsia
4808    Documentary obsession camcorder crush dream gi...
Name: tags, Length: 4809, dtype: object

In [31]:
df

,movie_id,title,overview,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction cultu...
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action ocean drug abuse exot...
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,Action Adventure Crime spy based on novel secr...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,Action Crime Drama Thriller dc comics crime fi...
4,49529,John Carter,"John Carter is a war-weary, former military ca...",Action Adventure Science Fiction based on nove...
...,...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,Action Crime Thriller united states–mexico bar...
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,Comedy Romance Edward Burns Kerry Bishé Marsha...
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",Comedy Drama Romance TV Movie date love at fir...
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,Daniel Henney Eliza Coupe Bill Paxton Daniel Hsia


In [32]:
df.tags = df.tags.apply(lambda x : x.lower())

In [33]:
df.tags[0]

'action adventure fantasy science fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d sam worthington zoe saldana sigourney weaver james cameron'

In [34]:
df

,movie_id,title,overview,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",action adventure fantasy science fiction cultu...
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",adventure fantasy action ocean drug abuse exot...
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,action adventure crime spy based on novel secr...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,action crime drama thriller dc comics crime fi...
4,49529,John Carter,"John Carter is a war-weary, former military ca...",action adventure science fiction based on nove...
...,...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,action crime thriller united states–mexico bar...
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,comedy romance edward burns kerry bishé marsha...
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",comedy drama romance tv movie date love at fir...
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,daniel henney eliza coupe bill paxton daniel hsia


In [35]:
tfidf = TfidfVectorizer(stop_words = 'english')

In [36]:
tfidf_matrix = tfidf.fit_transform(df.tags)

In [37]:
tfidf_matrix

<4809x14665 sparse matrix of type '<class 'numpy.float64'>'
	with 100104 stored elements in Compressed Sparse Row format>

In [38]:
tfidf_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [39]:
tfidf.get_feature_names_out()

array(['11', '15th', '16th', ..., '张立', '绝地奶霸', '超级妈妈'], dtype=object)

**COSINE SIMILARITY IS USED TO MEASURE SIMILARITY BETWEEN ITEMS**

In [41]:
cosine_sim = cosine_similarity(tfidf_matrix,tfidf_matrix)

**LETS APPLY THE COSINE SIMILARITY FOR THE WHOLE DF AND SEE**

In [43]:
eg = 'The Dark Knight Rises'

In [44]:
df[df['title'] == eg]

,movie_id,title,overview,tags
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,action crime drama thriller dc comics crime fi...


In [45]:
idx = df[df['title'] == eg].index[0]
idx

3

In [46]:
df.loc[3]

movie_id                                                49026
title                                   The Dark Knight Rises
overview    Following the death of District Attorney Harve...
tags        action crime drama thriller dc comics crime fi...
Name: 3, dtype: object

In [47]:
ss =list(enumerate(cosine_sim[idx]))
ss

[(0, 0.003555204849903463),
 (1, 0.02742755170331029),
 (2, 0.0533910260421415),
 (3, 1.0000000000000002),
 (4, 0.003739952408458775),
 (5, 0.04187419459158533),
 (6, 0.019276374948280273),
 (7, 0.04713008855848863),
 (8, 0.0),
 (9, 0.14380243705615997),
 (10, 0.08439247834594782),
 (11, 0.06511230821914027),
 (12, 0.004194169286178631),
 (13, 0.003628804449932671),
 (14, 0.12569376639441682),
 (15, 0.019032398839374066),
 (16, 0.045373107110874576),
 (17, 0.004484763398758831),
 (18, 0.04986000297489471),
 (19, 0.004570502612416563),
 (20, 0.06533420340243516),
 (21, 0.004716384287309233),
 (22, 0.0),
 (23, 0.0),
 (24, 0.00971777226384551),
 (25, 0.059852538295875085),
 (26, 0.054149336479335244),
 (27, 0.01989033163465751),
 (28, 0.0071674204006707334),
 (29, 0.06264443520916951),
 (30, 0.04815631880744118),
 (31, 0.04133118789189669),
 (32, 0.0),
 (33, 0.010049936883040672),
 (34, 0.0),
 (35, 0.050831235667504755),
 (36, 0.04753743542133695),
 (37, 0.0),
 (38, 0.004943181005995444),

In [48]:
ss = sorted(ss,key = lambda x: x[1],reverse = True)
ss

[(3, 1.0000000000000002),
 (65, 0.4467781458465154),
 (119, 0.38231390205367316),
 (1360, 0.23589894201430384),
 (210, 0.22691069656853657),
 (428, 0.2161568342789014),
 (1361, 0.2151037683830919),
 (1197, 0.1993752700175258),
 (303, 0.17515202302135868),
 (4644, 0.17275642588369727),
 (72, 0.15827613662907786),
 (299, 0.14643383785130554),
 (1742, 0.14573881239137965),
 (1011, 0.1448980798284734),
 (9, 0.14380243705615997),
 (813, 0.1411306030563322),
 (606, 0.13734731409242867),
 (1044, 0.1343842658843055),
 (163, 0.12892730037801123),
 (1722, 0.12765985645346414),
 (14, 0.12569376639441682),
 (238, 0.12397740167829284),
 (387, 0.11792327163597054),
 (870, 0.11589892290881915),
 (1297, 0.11360942641257468),
 (2795, 0.11336043156335092),
 (3824, 0.11297504544333284),
 (478, 0.1129487516195228),
 (339, 0.11148645560694019),
 (4272, 0.10928358237933726),
 (1150, 0.10532002164938811),
 (2386, 0.10430717703352893),
 (3175, 0.10381965898105162),
 (950, 0.10358675228455683),
 (625, 0.103550

In [49]:
def get_recommendations(title,cosine_sim = cosine_sim):
    if title not in df['title'].values:
        return f"❌ '{title}' not found in dataset."
    idx = df[df['title'] == title].index[0]
    sim_score = list(enumerate(cosine_sim[idx]))
    sim_score = sorted(sim_score,key = lambda x: x[1],reverse = True)
    sim_score = sim_score[1:11] # First 10 similar movie title
    movie_indices = [i[0] for i in sim_score]
    return df['title'].iloc[movie_indices]

In [50]:
print(get_recommendations('The Dark Knight Rises'))

65               The Dark Knight
119                Batman Begins
1360                      Batman
210               Batman & Robin
428               Batman Returns
1361                      Batman
1197                The Prestige
303                     Catwoman
4644    Amidst the Devil's Wings
72                 Suicide Squad
Name: title, dtype: object


In [51]:
print(get_recommendations('Alaipayuthey'))

❌ 'Alaipayuthey' not found in dataset.


In [52]:
with open('movie_data.pkl','wb') as f:
    pickle.dump((df,cosine_sim),f)